# <center> <font color = 'rebeccapurple'>Stochastic Watershed Model: Generating Stochastic Streamflows for the Shasta River Under 1 Degree of Warming</font> </center>

This Jupyter Notebook contains code for a stochastic watershed model (SWM) that can be used to develop an ensemble of stochastic streamflows for a single site. This code was used to produce the results found in **Shabestanipour et al. (2024)** for the Shasta River basin. With modifications to the input data and the specific ARMA model used (more details below), this code could be applied to other non-regulated river basins. 

This notebook is split into two parts: 1. SWM (generating stochastic streamflows) and 2. Diagnostics (assessing stochastic model performance).

***

**Computational Requirements**: We used Jupyter version 4.7.1 and Python version 3.8.8 to run the below code. Lower versions may result in error messages.

The code is somewhat computationally-intesnive. We found that allocating 32GB of memory allowed the code to run in full without crashing the kernel. 

The code blocks that will take the longest are generating the stochastic residuals in Part 1 and generating the figure inputs in Part 2 (depending on how many stochastic realizations you do). These should each take approx. 10 min maximum. Most other code blocks run very quickly. The code to generate AIC/BIC for various ARIMA models to determine which to use may also take some time (by default this code block is commented out).

***

# <center> <font color = 'red'> Part 1: SWM </font> </center>

## <font color='green'> Import Statements </font>

Here we import SWM functions (developed by Ghazal Shabestanipour) as well as existing external Python packages used in the notebook. Be sure that the file **Func_Lib.py** is contained in the same directory as this Jupyter Notebook.
For existing Python packages, if you do not have a particular package installed with your version of Python, you can install it using pip or conda (if using anaconda). Instructions for doing so can be found here:<br>
https://packaging.python.org/en/latest/tutorials/installing-packages/ (pip)<br>
https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/installing-with-conda.html (conda)

In [1]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

## <font color='green'> Load Input Data </font>

The only input data required is a csv containing daily streamflow data (observed and modeled). The columns of the csv are $date$, observed streamflow $Qgage$, deterministic model of streamflow $Qmodel$, and the difference between Qgage and Qmodel for each date $diff$.

For the Shasta, we have values from 10/01/1987 - 09/30/2018.

In [2]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [3]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-11.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.009027,0.418047,83.523228
1,1,1987-10-02,0.008879,0.417384,83.390695
2,2,1987-10-03,0.008716,0.416655,83.245014
3,3,1987-10-04,0.008566,0.415984,83.110959
4,4,1987-10-05,0.008399,0.415228,82.960006
...,...,...,...,...,...
11318,11318,2018-09-26,0.012813,0.387571,77.434302
11319,11319,2018-09-27,0.012443,0.385868,77.093989
11320,11320,2018-09-28,0.012103,0.384293,76.779327
11321,11321,2018-09-29,0.015773,0.400636,80.044574


## <font color='green'> Fit ARMA Model to Data </font>

Here we fit an AR4 model to our "lambdas" to get the model residuals and parameters.

In [4]:
p=4 #order of AR (autocorrelation with lag 4)
d=0 #order of I (no trend)
q=0 #order of MA (no moving average component)

mod = ARIMA(data['lambda'], order=(p,d,q)) #set up AR4 model using lambdas as input
res = mod.fit() #fit model

In [5]:
res.params #display the model parameters

const     0.023440
ar.L1     0.165057
ar.L2     0.187447
ar.L3     0.147261
ar.L4     0.137038
sigma2    0.067909
dtype: float64

Since we fit an AR4 model for this data, we have five parameters: a coefficient for each AR component (L1, L2, L3, and L4) and a constant term.

In [6]:
#create a column in data for the model residuals associated with each time step
data['Ar_res']=res.resid
data.head(5)

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037


## <font color = 'green'> Generate Random AR Parameter Sets </font>

We generate $m$ random AR parameter sets (called "Beta") to consider the uncertainty in the AR model parameters.

We use the Numpy function **random.multivariate_normal** to generate the parameter sets. This function samples the parameters from a multivariate normal distribution. The inputs to the function are the mean **(res.params)**, the covariance **(res.cov_params)**, and the number of parameter sets to generate. 

In [7]:
# m is the number of random AR parameter sets, Beta, generated for considering AR model uncertainty.
m = 100

# older version of ARIMA function
#Beta=np.random.multivariate_normal(np.array(res.params), np.array(res.cov_params()), m)

# updated Beta for using newer version of statsmodels ARIMA code
# Beta=np.random.multivariate_normal(np.array(res.params[:-1]), np.array(res.cov_params())[:-1,:-1], m)

# for reproducible Betas, use scipy.stats multivariate_normal function instead, setting random seed
Beta = stats.multivariate_normal(np.array(res.params[:-1]),np.array(res.cov_params())[:-1,:-1], seed=16)
Beta = Beta.rvs(m)

## <font color = 'green'> Generate Stochastic Residuals Using Simple Bootstrap </font>

There are 3 steps to generating the stochastic streamflow realizations:<br>

1. Generate stochastic residuals (use either simple bootstrap, conditional bootstrap, or k-Nearest Neighbors bootstrap)
2. Generate stochastic log errors by combining AR model uncertainty + stochastic residuals in AR4 model
3. Generate stochastic streamflow by transforming from log-ratios back to streamflows, including bias correction factor<br>

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 1: Generate Stochastic Residuals <green>

In [8]:
#now find the nearest Qmodel and residual associated with Qmodel for each flow value in Qfut
df_fut['Qmodel_nearest'] = 0 #initialize empty column for nearest Qmodel
df_fut['residual_nearest'] = 0 #initialize empty column for nearest Qmodel residual

for i in range(len(df_fut)): #loop through each row of df_fut
    #for each flow, calculate absolute difference of that flow and all flows in Qmodel
    #take absolute value and then find the index of the minimum absolute value
    #populate Qmodel_nearest column with data['Qmodel'] at this index
    #and residual_nearest column with data['Ar_res'] at this index
    df_fut['Qmodel_nearest'].iloc[i] = data['Qmodel'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    df_fut['residual_nearest'].iloc[i] = data['Ar_res'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    
#now use KNN bootstrapping as before, with Epsilon defined as the Qmodel_nearest and residual_nearest columns from df_fut
#basically exactly what we had before, with some data "repeats" to correspond to the new data points
#and in the correct order to match Qfut

n = 100 # n is the number of stochastic streamflow realizations generated for each Beta set
Epsilon = df_fut.filter(['residual_nearest','Qmodel_nearest'])
Epsilon = np.array(Epsilon) # turn dataframe into numpy array

E = [] #initialize empty list to store errors
for t in range(0,len(df_fut)): # loop through all time periods
    E.append(Epsilon - [0,Epsilon[t,1]])  # first column residual, second column diff between flow and all other flows

k = 750

Er = np.zeros((len(df_fut),k)) #initialize empty 2d array with dimensions time periods x k (# nearest neighbors)
for t in range(0,len(df_fut)): #loop through all time periods
    E[t][:,1] = np.abs(E[t][:,1]) #get absolute values of errors
    x = E[t][np.argsort(E[t][:, 1])] #sort errors by magnitude
    Er[t,:] = x[0:k,0]
    

R = np.zeros((len(df_fut),n*m)) #initialize empty 2d array with dimensions time periods x # realizations
for i in range(0,n*m): #loop through all realizations
    I = np.random.randint(low=0, high=k, size=len(df_fut)) #randomly identify indices
    R[:,i] = Er[tuple(np.arange(0,len(df_fut),1)),I] #sample from Er at random indices I
    
E = R #set E to equal R

/cluster/tufts/hpc/tools/anaconda/202105/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


One needs to experiment with different k values to find the appropriate k for each basin by going trough all the next steps of the SWM and diagnostic figures.  

### <font color='green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 2: Generate Stochastic Log Errors </font>

In [9]:
# l is a matrix of [len(data),m*n] representing stochastic log-ratio errors         
L = np.zeros((len(df_fut),m*n)) #initialize empty array to populate below

# for the first five time periods, we set values to be the deterministic model lambdas across all realizations
# this is because we do not have enough time periods in the past to populate the AR4 model stochastically
for i in range(0,m*n):
    L[0:5,i] = data['lambda'].iloc[0:5] 

for i in range(0,m):    
    B = Beta[i,:] #let B = random parameter set for realization i
    for t in range(5,len(df_fut)):
        # L(t) = B0 + B1*L[t-1] + B2*L[t-2] + B3*L[t-3] + B4*L[t-4] + E[t]
        L[t, i*n:((i+1)*n)] = B[0] + \
                              B[1] * L[t-1, i*n:((i+1)*n)] + \
                              B[2] * L[t-2, i*n:((i+1)*n)] + \
                              B[3] * L[t-3, i*n:((i+1)*n)] + \
                              B[4] * L[t-4, i*n:((i+1)*n)] + \
                              E[t, i*n:((i+1)*n)]

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 3: Generate Stochastic Streamflows </font>

In [10]:
# Q is a matrix of stochastically simulated streamflows of size [m*n, len(data)]
# Just like lambda = log(Qmodel/Qgage), we now set Q = Qmodel/e^L where
# L is stochastic lambdas and Q is stochastic streamflow
Q = np.array(df_fut['Qfut']) / np.exp(L.T)

# BCF is transformation bias correction factor 
# this address the bias introduced by moving to log space
BCF = 1 / np.exp(-np.mean(data['lambda']) + np.var(data['lambda'])/2)

Q = Q*BCF

In [11]:
print(BCF)

0.9807641078478005


In [12]:
display(pd.DataFrame(Q))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.405513,11.109659,14.823923,15.454234,24.145252,28.658974,35.552429,42.646712,22.411076,84.122860,...,72.731751,69.286934,122.744551,79.434497,94.781076,92.169474,92.224024,109.157231,210.674254,125.429151
1,13.405513,11.109659,14.823923,15.454234,24.145252,35.162399,34.372202,36.543279,44.562788,49.861673,...,71.371475,64.621833,56.873103,67.150561,60.891159,15.400653,62.392757,58.158869,78.737367,69.869092
2,13.405513,11.109659,14.823923,15.454234,24.145252,27.160041,30.536985,31.932813,41.308617,44.723581,...,82.872674,64.454746,76.259689,82.348522,81.890872,77.727432,96.608756,84.977187,76.156137,133.992138
3,13.405513,11.109659,14.823923,15.454234,24.145252,6.697362,27.094737,27.193340,32.242063,32.466139,...,15.811258,68.623820,65.738015,68.578278,53.723188,57.153304,65.112448,60.543354,66.254591,102.407231
4,13.405513,11.109659,14.823923,15.454234,24.145252,19.896898,35.322935,17.942524,40.240947,35.278173,...,89.792469,135.404960,100.226313,86.534691,129.624426,107.335908,110.451724,100.849628,98.358593,126.352152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.405513,11.109659,14.823923,15.454234,24.145252,26.284681,38.072041,33.889075,48.060582,74.279421,...,72.727985,64.827811,71.057109,62.395939,84.425292,60.442238,81.473094,65.145391,79.797318,100.324219
9996,13.405513,11.109659,14.823923,15.454234,24.145252,33.443981,40.650174,35.850868,22.776496,46.782095,...,71.956549,105.249221,72.566262,87.686583,62.177105,77.062358,63.799228,72.067267,21.074972,81.202434
9997,13.405513,11.109659,14.823923,15.454234,24.145252,33.226969,53.305199,34.492757,51.053144,42.937778,...,67.992872,85.652099,82.492779,71.088218,75.536685,76.340121,68.180481,59.006946,70.620123,77.450668
9998,13.405513,11.109659,14.823923,15.454234,24.145252,29.593194,60.611981,40.923353,63.512823,73.970053,...,56.315321,44.567065,62.225438,70.286007,48.940074,47.005771,54.943083,53.788228,70.075059,110.323491


In [13]:
display(pd.DataFrame(L.T))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,1.810035,1.996299,1.706123,1.66287,1.214848,1.041807,0.824631,0.641532,1.283541,-0.040891,...,0.061480,0.105510,-0.469563,-0.037084,-0.217235,-0.193622,-0.198619,-0.371277,-0.987153,-0.239015
1,1.810035,1.996299,1.706123,1.66287,1.214848,0.837296,0.858392,0.795985,0.596197,0.482135,...,0.080359,0.175214,0.299720,0.130912,0.225247,1.595597,0.192153,0.258334,-0.002957,0.346102
2,1.810035,1.996299,1.706123,1.66287,1.214848,1.095527,0.976701,0.930848,0.672025,0.590887,...,-0.069048,0.177803,0.006398,-0.073111,-0.071053,-0.023202,-0.245067,-0.120871,0.030375,-0.305055
3,1.810035,1.996299,1.706123,1.66287,1.214848,2.495560,1.096300,1.091510,0.919824,0.911190,...,1.587535,0.115126,0.154865,0.109873,0.350490,0.284269,0.149486,0.218152,0.169656,-0.036232
4,1.810035,1.996299,1.706123,1.66287,1.214848,1.406710,0.831107,1.507309,0.698211,0.828123,...,-0.149243,-0.564504,-0.266888,-0.122696,-0.530306,-0.345957,-0.378977,-0.292118,-0.225460,-0.246347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.810035,1.996299,1.706123,1.66287,1.214848,1.128287,0.756160,0.871389,0.520634,0.083554,...,0.061531,0.172031,0.077059,0.204349,-0.101532,0.228318,-0.074671,0.144891,-0.016330,-0.015681
9996,1.810035,1.996299,1.706123,1.66287,1.214848,0.887402,0.690637,0.815114,1.267367,0.545887,...,0.072195,-0.312565,0.056043,-0.135920,0.204348,-0.014608,0.169861,0.043912,1.315074,0.195781
9997,1.810035,1.996299,1.706123,1.66287,1.214848,0.893912,0.419606,0.853733,0.460229,0.631636,...,0.128855,-0.106528,-0.072168,0.073928,0.009716,-0.005192,0.103444,0.243857,0.105845,0.243085
9998,1.810035,1.996299,1.706123,1.66287,1.214848,1.009729,0.291147,0.682781,0.241855,0.087727,...,0.317291,0.546771,0.209779,0.085276,0.443738,0.479736,0.319304,0.336457,0.113593,-0.110691


In [14]:
data

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037
...,...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934,-0.022115
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129,-0.024599
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849,0.261246
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763,-0.024250


## <font color = 'green'> Save Outputs </font>

Outputs saved as **.npy** files. The outputs are:

1. E.npy: stochastic residuals
2. L.npy: stochastic log ratios
3. Q.npy: stochastic streamflows

In [15]:
# save the results as .npy files
np.save('Q_1degree.npy',Q)
np.save('StochasticE_1degree.npy',E)
np.save('L_1degree.npy',L)

***

# <center> <font color = 'red'> Part 2: Diagnostic Figures </font> </center>

Here, we include code to generate T-year flood flows for the specified warming scenario.

## <font color = 'green'> Reloading in the data </font>

In [16]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

In [17]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [18]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-11.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.009027,0.418047,83.523228
1,1,1987-10-02,0.008879,0.417384,83.390695
2,2,1987-10-03,0.008716,0.416655,83.245014
3,3,1987-10-04,0.008566,0.415984,83.110959
4,4,1987-10-05,0.008399,0.415228,82.960006
...,...,...,...,...,...
11318,11318,2018-09-26,0.012813,0.387571,77.434302
11319,11319,2018-09-27,0.012443,0.385868,77.093989
11320,11320,2018-09-28,0.012103,0.384293,76.779327
11321,11321,2018-09-29,0.015773,0.400636,80.044574


In [19]:
Q = np.load('Q_1degree.npy')
m=100
n=100

In [20]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.405513,11.109659,14.823923,15.454234,24.145252,28.658974,35.552429,42.646712,22.411076,84.122860,...,72.731751,69.286934,122.744551,79.434497,94.781076,92.169474,92.224024,109.157231,210.674254,125.429151
1,13.405513,11.109659,14.823923,15.454234,24.145252,35.162399,34.372202,36.543279,44.562788,49.861673,...,71.371475,64.621833,56.873103,67.150561,60.891159,15.400653,62.392757,58.158869,78.737367,69.869092
2,13.405513,11.109659,14.823923,15.454234,24.145252,27.160041,30.536985,31.932813,41.308617,44.723581,...,82.872674,64.454746,76.259689,82.348522,81.890872,77.727432,96.608756,84.977187,76.156137,133.992138
3,13.405513,11.109659,14.823923,15.454234,24.145252,6.697362,27.094737,27.193340,32.242063,32.466139,...,15.811258,68.623820,65.738015,68.578278,53.723188,57.153304,65.112448,60.543354,66.254591,102.407231
4,13.405513,11.109659,14.823923,15.454234,24.145252,19.896898,35.322935,17.942524,40.240947,35.278173,...,89.792469,135.404960,100.226313,86.534691,129.624426,107.335908,110.451724,100.849628,98.358593,126.352152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.405513,11.109659,14.823923,15.454234,24.145252,26.284681,38.072041,33.889075,48.060582,74.279421,...,72.727985,64.827811,71.057109,62.395939,84.425292,60.442238,81.473094,65.145391,79.797318,100.324219
9996,13.405513,11.109659,14.823923,15.454234,24.145252,33.443981,40.650174,35.850868,22.776496,46.782095,...,71.956549,105.249221,72.566262,87.686583,62.177105,77.062358,63.799228,72.067267,21.074972,81.202434
9997,13.405513,11.109659,14.823923,15.454234,24.145252,33.226969,53.305199,34.492757,51.053144,42.937778,...,67.992872,85.652099,82.492779,71.088218,75.536685,76.340121,68.180481,59.006946,70.620123,77.450668
9998,13.405513,11.109659,14.823923,15.454234,24.145252,29.593194,60.611981,40.923353,63.512823,73.970053,...,56.315321,44.567065,62.225438,70.286007,48.940074,47.005771,54.943083,53.788228,70.075059,110.323491


## <font color = 'green'> Generating Figure Inputs </font>

In [21]:
def annualmax(Q,data,n):
    annual_max=np.array(pd.DataFrame(Q.T,index=data['date']).resample('Y').max())
    annualmaxsort = np.zeros(annual_max.shape)
    for i in range(0,n):
        annualmaxsort[:,i]=np.sort(annual_max[:,i])[::-1]
    
    annualmaxsort = pd.DataFrame(annualmaxsort)
    #annualmaxsort['rank']=(annualmaxsort.index) + 1
    #annualmaxsort['Exeedance']=annualmaxsort['rank']/(len(annualmaxsort)+1)
    #annualmaxsort['non_Exceedance']=1-annualmaxsort['Exeedance']
    #Data=data.set_index('date')
    #annualmaxobs=Data['Qgage'].resample('Y').max()
    #annualmaxmodel=Data['Qmodel'].resample('Y').max()
    #annualmaxsort['obs']=np.sort(annualmaxobs)[::-1]
    #annualmaxsort['model']=np.sort(annualmaxmodel)[::-1]
    amax=np.array(annualmaxsort)[:,0:n]
    return annualmaxsort
Annualmax = annualmax(Q, df_fut, m*n)

In [22]:
# calculating 50-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/50)
flood50Y1=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood50Y1 = np.array(flood50Y1)

In [23]:
# calculating 100-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/100)
flood100Y1=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood100Y1 = np.array(flood100Y1)

In [24]:
# calculating 500-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/500)
flood500Y1=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood500Y1 = np.array(flood500Y1)

In [25]:
np.save('flood50Y1.npy',flood50Y1)
np.save('flood100Y1.npy',flood100Y1)
np.save('flood500Y1.npy',flood500Y1)